In [1]:
from gin_train.trainers import KerasTrainer
from kipoi_cadd.data import CaddDataset
from kipoi_cadd.data import cadd_train_valid_data, cadd_serialize_string_row
from kipoi_cadd.data import save_variant_ids
from kipoi_cadd.data import train_test_split_indexes
from kipoi_cadd.config import get_data_dir
import pandas as pd
import numpy as np
from random import shuffle
from kipoi_cadd.models import logistic_regression_scikit, logistic_regression_keras
from gin_train.metrics import ClassificationMetrics
from sklearn.model_selection import train_test_split
%load_ext line_profiler

Using TensorFlow backend.


In [2]:
# Create train, test and validation indexes
all_variant_id_file = "/s/project/kipoi-cadd/data/raw/v1.3/training_data/variant_ids.pkl"
training_data_dir = "/s/project/kipoi-cadd/data/raw/v1.3/training_data/"
train_idx_path = training_data_dir + "train_idx.pkl"
test_idx_path = training_data_dir + "test_idx.pkl"
valid_idx_path = training_data_dir + "valid_idx.pkl"
lmdb_dir = training_data_dir + "lmdb"

In [3]:
ds2 = CaddDataset(lmdb_dir, train_idx_path)
# it = ds2.batch_iter()

In [24]:
%lprun -f ds2.__getitem__ ds2[100]

Timer unit: 1e-06 s

Total time: 0.000289 s
File: /data/ouga04b/ag_gagneur/project_local/kipoi-cadd/kipoi_cadd/data.py
Function: __getitem__ at line 43

Line #      Hits         Time  Per Hit   % Time  Line Contents
    43                                               def __getitem__(self, idx):
    44         1          2.0      2.0      0.7          if self.lmdb_cadd is None:
    45                                                       self.lmdb_cadd = lmdb.open(self.lmdb_cadd_path, readonly=True, lock=False)
    46                                                       self.txn = self.lmdb_cadd.begin()
    47                                           
    48                                                   # TODO: Make this distiction clearer, do we want to search by loc or iloc?
    49                                                   # Loc breaks when a splitted dataset doesn't have idx.
    50                                                   # variant_id = self.variant_ids.loc[i

In [4]:
b = next(it)

In [5]:
from tqdm import tqdm

In [6]:
m = logistic_regression_keras(n_features=1063)

In [7]:
prev_weights = None
prev_batch = None
for b in tqdm(it, total=len(ds2)//32):
    if np.any(np.isnan(b['targets'])):
        print("Nan value in targets")
        break
    if np.any(np.isnan(b['inputs'])):
        print("Nan value in inputs")
        break
    m.train_on_batch(b['inputs'], b['targets'])
    
    if np.any(np.isnan(m.get_weights()[0])):
        print("Nan value in weights")
        break
    prev_weights = m.get_weights()[0]

  0%|          | 1894/657057 [00:16<1:44:42, 104.29it/s]

Nan value in weights


In [ ]:
inf

In [35]:
np.sum(b['inputs'] == np.inf, axis=0).argmax()

124

In [39]:
b['metadata']['variant_id'][10]

"11:2649934:A:['G']"

In [44]:
# Find fucked up variant
inputfile = training_data_dir + "training_data.tsv"
row_example = pd.read_csv(inputfile,
                              sep='\t',
                              nrows=1,
                              usecols=[0,1,2,3])
row_example

,y,Chrom,Pos,Ref
0,0,1,379177,T


In [57]:
with open(inputfile, 'r') as f:
    counter = 0
    for line in f:
        if counter == 0: print(line)
        if "\t2649934" in line:
            print("Line number: "+str(counter)+"\n",line, "\n")
        counter += 1

y	Chrom	Pos	Ref	Alt	Type	Length	isTv	Consequence	GC	CpG	mapAbility20bp	mapAbility35bp	scoreSegDup	priPhCons	mamPhCons	verPhCons	priPhyloP	mamPhyloP	verPhyloP	GerpN	GerpS	GerpRS	GerpRSpval	bStatistic	mutIndex	dnaHelT	dnaMGW	dnaProT	dnaRoll	mirSVR-Score	mirSVR-E	mirSVR-Aln	targetScan	fitCons	cHmmTssA	cHmmTssAFlnk	cHmmTxFlnk	cHmmTx	cHmmTxWk	cHmmEnhG	cHmmEnh	cHmmZnfRpts	cHmmHet	cHmmTssBiv	cHmmBivFlnk	cHmmEnhBiv	cHmmReprPC	cHmmReprPCWk	cHmmQuies	EncExp	EncH3K27Ac	EncH3K4Me1	EncH3K4Me3	EncNucleo	EncOCC	EncOCCombPVal	EncOCDNasePVal	EncOCFairePVal	EncOCpolIIPVal	EncOCctcfPVal	EncOCmycPVal	EncOCDNaseSig	EncOCFaireSig	EncOCpolIISig	EncOCctcfSig	EncOCmycSig	Segway	tOverlapMotifs	motifDist	motifECount	motifEHIPos	motifEScoreChng	TFBS	TFBSPeaks	TFBSPeaksMax	DAF	minDistTSS	minDistTSE	cDNApos	relcDNApos	CDSpos	relCDSpos	protPos	relProtPos	Domain	Dst2Splice	Dst2SplType	oAA	nAA	Grantham	PolyPhenCat	PolyPhenVal	SIFTcat	SIFTval	mirSVR-Score.na	targetScan.na	cDNApos.na	CDSpos.na	protPos.na	Grantham.na	Pol

In [58]:
inputfile2 = training_data_dir + "training_data.imputed.csv"
with open(inputfile2, 'r') as f:
    counter = 0
    for line in f:
        if counter == 0: print(line)
        if counter == 9652843:
            print("Line number: "+str(counter)+"\n",line, "\n")
            break
        counter += 1
-1 1 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 1

y,RefxA,RefxC,RefxG,RefxT,RefxN,AltxA,AltxC,AltxG,AltxT,AltxN,TypexSNV,TypexINS,TypexDEL,Length,isTv,ConsequencexU3,ConsequencexU5,ConsequencexDN,ConsequencexIG,ConsequencexI,ConsequencexNC,ConsequencexIF,ConsequencexFS,ConsequencexNS,ConsequencexR,ConsequencexCS,ConsequencexS,ConsequencexSG,ConsequencexSL,ConsequencexSN,ConsequencexO,ConsequencexUP,GC,CpG,priPhCons,mamPhCons,verPhCons,priPhyloP,mamPhyloP,verPhyloP,GerpN,GerpS,GerpRS,GerpRSpval,bStatistic,mutIndex,dnaHelT,dnaMGW,dnaProT,dnaRoll,mirSVRs,IND_mirSVRs,mirSVRe,mirSVRa,targetScan,IND_targetScan,cHmmTssA,cHmmTssAFlnk,cHmmTxFlnk,cHmmTx,cHmmTxWk,cHmmEnhG,cHmmEnh,cHmmZnfRpts,cHmmHet,cHmmTssBiv,cHmmBivFlnk,cHmmEnhBiv,cHmmReprPC,cHmmReprPCWk,cHmmQuies,EncExp,EncH3K27Ac,EncH3K4Me1,EncH3K4Me3,EncNucleo,EncOCC,EncOCCombPVal,EncOCDNasePVal,EncOCFairePVal,EncOCpolIIPVal,EncOCctcfPVal,EncOCmycPVal,EncOCDNaseSig,EncOCFaireSig,EncOCpolIISig,EncOCctcfSig,EncOCmycSig,EncSegwayxC0,EncSegwayxC1,EncSegwayxD,EncSegwayxE/GM,EncSegwayxF0,EncSegwa

In [64]:
import pyarrow as pa
ser = cadd_serialize_string_row(line, variant_id="11:2649934:A:['G']", separator=',')
deser = pa.deserialize(ser.to_buffer())
print(deser)

{'inputs': array([1., 0., 0., ..., 0., 0., 0.], dtype=float16), 'targets': -1.0, 'metadata': {'variant_id': "11:2649934:A:['G']"}}


In [68]:
line

'-1,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.30,0.01,0.073,0.074,0.097,0.140,0.142,0.152,0.131,0.131,0,0,982,14,-2.72,0.24,2.27,-1.17,0,1,0,0,0,1,0,0,0,0,0.087,0,0,0.024,0.252,0,0,0,0.031,0.102,0.504,0,1.20,2.00,2.00,1.90,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,11.1745,9.9221,71291,0,0.78,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,982,71291,0,0,0.131,0.131,0.074,0.142,9.9221,11.1745,0.073,0.140,0,0.78,0,0,0.097,0.152,14,-2.72,0.24,2.27,-1.17,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [75]:
np.array(71291/1000, dtype=np.float16)

array(71.3, dtype=float16)

In [69]:
np.array(71291, dtype=np.float16)

array(inf, dtype=float16)

In [67]:
deser['inputs']

inf

In [38]:
b['inputs'][:, 124].argmax()

10

In [27]:
b['inputs'].max()

inf

In [65]:
b['inputs'].dtype

dtype('float16')

In [24]:
b['targets'].max()

1.0

In [13]:
np.isnan(m.get_weights()[0]).sum()

2

In [17]:
np.argmax(np.isnan(m.get_weights()[0]))

124

In [19]:
prev_weights[124]

array([0.17442241], dtype=float32)

In [18]:
m.get_weights()[0][124]

array([nan], dtype=float32)

In [12]:
prev_weights.max()

0.17442241

In [11]:
prev_weights.min()

-0.13959962

In [9]:
m.get_weights()

[array([[ 4.4871857e-03],
        [ 5.3848815e-04],
        [ 5.0784983e-03],
        ...,
        [ 2.5191470e-36],
        [ 5.3655251e-36],
        [-1.6577878e-36]], dtype=float32), array([-0.05025654], dtype=float32)]

In [8]:
prev_weights

array([[ 3.8092749e-03],
       [ 3.6265361e-03],
       [ 3.9158156e-03],
       ...,
       [-2.5431340e-36],
       [ 5.3655251e-36],
       [ 9.8438798e-36]], dtype=float32)

  0%|          | 1894/657057 [00:30<1:44:42, 104.29it/s]

In [8]:
b['inputs']

array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]], dtype=float16)

  0%|          | 1885/657057 [00:30<1:14:30, 146.56it/s]

In [62]:
b['targets'].shape

(32,)

In [63]:
b['targets']

array([0., 0., 1., 1., 1., 0., 1., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0.,
       1., 1., 1., 0., 1., 0., 0., 1., 1., 1., 0., 0., 0., 0., 1.])

In [67]:
import numpy as np

In [68]:
np.any(b['inputs'] == np.nan)

False

In [64]:
b['inputs'].shape

(32, 1063)

In [78]:
it = ds2.batch_train_iter()

In [79]:
next(it)

(array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.]], dtype=float16),
 array([0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0.,
        1., 0., 0., 1., 1., 0., 0., 1., 1., 1., 0., 0., 1., 0., 1.]))

In [81]:
m.get_weights()

[array([[nan],
        [nan],
        [nan],
        ...,
        [nan],
        [nan],
        [nan]], dtype=float32), array([nan], dtype=float32)]

In [82]:
next(it)

(array([[1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]], dtype=float16),
 array([1., 1., 0., 0., 1., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 1.,
        0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 1., 1., 0., 0.]))

In [93]:
np.isnan(m.get_weights()[0][0][0])

True

In [84]:
np.any(m.get_weights()[0] == np.nan)

False

In [ ]:
for x,y in it:
    

In [80]:
m.fit_generator(it, steps_per_epoch=len(ds2)//32, epochs=1)

Epoch 1/1
  2379/657057 [..............................] - ETA: 1:17:53 - loss: nan - acc: 0.4028

KeyboardInterrupt: 

In [ ]:
%%time
lmdb_cadd = lmdb.Environment(lmdb_dir, map_size=800000000, lock=False)
txn = lmdb_cadd.begin(write=False, buffers=True)

variant_id = variant_ids[idx]
buf = bytes(self.txn.get(variant_id.encode('ascii')))

In [ ]:
%%time
ds2 = CaddDataset(lmdb_dir, train_idx_path)
dv2 = CaddDataset(lmdb_dir, valid_idx_path)
tr2 = KerasTrainer(logistic_regression_keras(n_features=1063), dt2, dv2, get_data_dir() + "/models/try10")
tr2.train(batch_size=512, epochs=10, num_wo